In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from functions import description_cleaner, vocabulary_creator, reverse_index_creator
nltk.download('stopwords')

restaurants_df = pd.read_csv("restaurants_i.tsv", sep="\t", header = 0)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Xxqas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
restournats_descriptions = restaurants_df.description[0:2]

parsed_descriptions = description_cleaner(restournats_descriptions)

ID_descritpion, vocab, tot_n_of_words = vocabulary_creator(parsed_descriptions)

reverse_index = reverse_index_creator(ID_descritpion)

pd.Series(vocab).to_csv("vocabulary.csv", index=True, encoding="utf-8", header= False)
